## Dependencies

In [1]:
import openai
import pandas as pd
from IPython.display import display, Markdown
import matplotlib.pyplot as plt
pd.set_option('display.max_colwidth', None)
from transformers import AutoModelForCausalLM, AutoTokenizer,T5Tokenizer, T5ForConditionalGeneration
import torch
from dotenv import load_dotenv
import os
load_dotenv()
openai.api_key = os.getenv("CHATGPT_API_KEY")
from huggingface_hub import login
login(token=os.getenv("HF_API_KEY"))

/home/hc4293/miniconda3/envs/imgsenv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Initialize sentences and tasks

In [2]:
# source: A speech by Abhraham Lincoln (Gettysburg Address, 1863).
sen_1= "The world will little note, nor long remember what we say here, but it can never forget what they did here."
# source: Wernicke's Aphasia from Aphasia Community INC.
sen_2= "You know that smoodle pinkered and that I want to get him round and take care of him like you want before."


tasks = [
    "constituency parse",
    "dependency parse",
    "CCG parse",
    "semantic roles",
    "abstract meaning representation (AMR)",
    "coreference resolution",
]

## Models

#### GPT4

In [3]:
# function to query ChatGPT
def query_chatgpt(sentence, task):
    prompt = (
        f"Perform the following task on the given sentence and only provide the desired output:\n"
        f"Task: {task}\n"
        f"Sentence: {sentence}\n"
    )
    try:
        response = openai.ChatCompletion.create(
            model="gpt-4",
            messages=[{"role": "user", "content": prompt}],
            max_tokens=1000
        )
        return response["choices"][0]["message"]["content"]
    except Exception as e:
        return str(e)

#### Pythia

In [4]:
# load pythia model and tokenizer
pythia_model_name = "EleutherAI/pythia-2.8b-deduped"
pythia_tokenizer = AutoTokenizer.from_pretrained(pythia_model_name)
pythia_tokenizer.pad_token = pythia_tokenizer.eos_token
pythia_model = AutoModelForCausalLM.from_pretrained(pythia_model_name)

def query_pythia(sentence, task, max_length=512):
    prompt = (
        f"Perform the following task on the given sentence and only provide the desired output:\n"
        f"Task: {task}\n"
        f"Sentence: {sentence}"
    )
    
    inputs = pythia_tokenizer(prompt, return_tensors="pt", padding=True)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    pythia_model.to(device)
    inputs = inputs.to(device)
    
    outputs = pythia_model.generate(
        inputs.input_ids,
        attention_mask=inputs.attention_mask,
        max_length=max_length,
        do_sample=True,
        pad_token_id=pythia_tokenizer.pad_token_id
    )
    response = pythia_tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    return response.replace(prompt, "").strip()


#### T5 small

In [5]:
t5_model_name = "t5-small"
t5_tokenizer = T5Tokenizer.from_pretrained(t5_model_name)
t5_tokenizer.pad_token = t5_tokenizer.eos_token
t5_model = T5ForConditionalGeneration.from_pretrained(t5_model_name)

def query_t5(sentence, task, max_length=512):
    prompt = (
        f"Perform the following task on the given sentence and only provide the desired output:\n"
        f"Task: {task}\n"
        f"Sentence: {sentence}"
    )
    
    inputs = t5_tokenizer(prompt, return_tensors="pt", padding=True)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    t5_model.to(device)
    inputs = inputs.to(device)
    
    outputs = t5_model.generate(
        inputs.input_ids,
        attention_mask=inputs.attention_mask,
        max_length=max_length,
        do_sample=True,
        pad_token_id=t5_tokenizer.pad_token_id
    )
    response = t5_tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    return response.replace(prompt, "").strip()


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [6]:
def process_and_display_results(sentences, tasks, model_name, query_function):
    result_dfs=[]
    for sentence in sentences:
        results = []
        for task in tasks:
            output = query_function(sentence, task)
            results.append({
                "Task": task,
                "Output": output
            })
        
        df_results = pd.DataFrame(results)
        result_dfs.append(df_results)
        
        display(Markdown(f"### Model: {model_name}"))
        display(Markdown(f"**Sentence:** {sentence}"))
        
        display(df_results)
    return result_dfs

## Outputs

In [7]:
pythia_sen1_df, pythia_sen2_df = process_and_display_results(
    sentences=[sen_1,sen_2],
    tasks=tasks,
    model_name="Pythia-2.8B",
    query_function=query_pythia
)

### Model: Pythia-2.8B

**Sentence:** The world will little note, nor long remember what we say here, but it can never forget what they did here.

,Task,Output
0,constituency parse,"The task will only provide {world, little}, {will, note}, {not, long}, {say, here}, {but, it}, etc.\nNote: Only add one of each token in the list provided\n\nA:\n\nRegular Expression\ngsub(/^(?:[^\[\]\(\)\-\.\.\s]*\[([^\]\(\)]*)|(\^\s*)\])/ig, ""\\1"", yourSentence)\n\nReplace the [^,^\[\]\(\)\-\.\.\s]* with this \K stuff. So, what all these stuff is doing is that it keeps the previous \K stuff and then replaces the \K stuff with what you want to replace it with. So, what this means is that, it is matching the non-space, non-comment, and non-digits at the beginning of the string up to the first [] or () that it can find. Then it creates a match that matches the whole of what it just matched plus everything that is after it. So, all \K stuff is replaced with the stuff that it matched \K the first thing that it matched. Here, \s* matches a whole bunch of characters without spaces. \s matches a sequence of characters that are not spaces.\nSo, what we do here, first, is match up to the first [] or () and say that I want what is after and I want what is before what I just matched \K. Then, once I get to that portion, I match everything that is after it. So, all of it. Now, the other stuff is this \K stuff. So, \K is \K \s and \K is everything that is matched and didn't get replaced. So, this matches all of the characters after what is the first thing that you matched \K and everything else that it matched. Then, it replaces it with the part that you want it to replace it with. So, that is an example of how \K stuff is treated and what it matches.\ngsub also returns the matched string, so it is optional in this example. In other examples, instead of \K, or \s, it would be passed in and you would do"
1,dependency parse,"Predicate: NLP -> Dependency Parsing: SubjParc\n\nThis will result in the following list of tuples:\n[(('will', 'little', None), None), # The part before the punctuation marks\n(('world', 'little', None), None), # The part after the punctuation marks\n((None, None), None), # The rest of the sentence"
2,CCG parse,"Return the number of words in the sentence in the returned list.\nThe task is given the sequence of the words (the sentence) as a sentence object.\n(return-type list)\n\nTask: CCG parse\nSentence: He gave up\n\nReturn the number of words in the sentence in the returned list.\nThe task is given the sequence of the words (the sentence) as a sentence object.\n(return-type list)\n\nTask: CCG parse\nSentence: It's so small\n\nReturn the number of words in the sentence in the returned list.\nThe task is given the sequence of the words (the sentence) as a sentence object.\n(return-type list)\n\nTask: Find a given string in between two characters\n\nGet the first character that matches the given string, after removing the specified number of characters. For example, you may find a string: ""the"" in ""the horse"".\nReturns nothing.\n(return-type string)\n\nTask: Find a given string within a sentence\n\nSearch through a sentence and find the first index where the matching string occurs, then returns a list containing the first character of the matching string, after removing this character from the word at that index. For example, you may find a string: ""the"" in the sentence: The horse.\nReturns nothing.\n(return-type list)\n\nTask: Find a given string within a sentence\n\nSearch through a sentence and find the first index where the matching string occurs, then returns a list containing the first character of the matching string, after removing this character from the word at that index. For example, you may find a string: ""the"" in the sentence: He gave up.\nReturns nothing.\n(return-type list)\n\nTask: Find a given string within a sentence\n\nSearch through a sentence and find the first index where the matching string occurs, then returns a list containing the first character of the matching string, after removing this character from the word at that index. For example, you may 

### Model: Pythia-2.8B

**Sentence:** You know that smoodle pinkered and that I want to get him round and take care of him like you want before.

,Task,Output
0,constituency parse,"My solution looks like\nString sentence = ""You know that smoodle pinkered and that I want to get him round and take care of him like you want before."";\nString[] tokens = sentence.split("" "");\n\nString keySet = sentence.substring(0, sentence.lastIndexOf("", "")+1);\nString[] tokenSet = new String[tokens.length];\n\nfor (int i=0; i < tokens.length; i++){\ntokenSet[i] = tokens[i];\n}\n\nString[] tokensArray1 = new String[tokenSet.length];\nfor (int x=0; x< keySet.length(); x++){\n for (int i=1; i<keySet.length(); i++){\n tokensArray1[x-1] = keySet.substring(x, i) + ""."" + tokenSet[i-1];\n }\n System.out.println(tokensArray1[0]); \n }\n\nKeySet can be made to be [before, after, during] and then used to give the token in the string. It is a good practice to first test the solution with some strings for validation. For example:\nString sentence = ""You know that smoodle pinkered and that I want to get him round and take care of him like you want before."", \nString keySet = ""before, during, after"",\nString tokenSet = ""before"";\nString []tokensArray1 = Tokenizer.tokenize(sentence, tokenSet);\nSystem.out.println(Arrays.toString(tokensArray1));\n\nShould output:\n[before, you know, during, that, I, want, to, get, him, round, and, take, care, like]"
1,dependency parse,"So, please let's start an\ndepreciated activity of mine.\nExample:\nInput: You know that smoodle pinkered and that I want to get him round and take care of him like you want before. So, please let's start\nan\ndepreciated activity of mine.\nDep:\ndependency[1]({\n ""depObject"": {\n ""target"": {\n ""arg0"": ""You know"",\n ""arg1"": ""that"",\n ""arg2"": ""smoodle"",\n ""arg3"": ""pinkered"",\n ""arg4"": ""and"",\n ""arg5"": ""that"",\n ""arg6"": ""I want"",\n ""arg7"": ""to get"",\n ""arg8"": ""him"",\n ""arg9"": ""round"",\n ""arg10"": ""and"",\n ""arg11"": ""take"",\n ""arg12"": ""care"",\n ""arg13"": ""of him"",\n ""arg14"": ""like"",\n ""arg15"": ""you""\n },\n ""left"": []\n }\n});\n\nOutput:\ndepreciated activity of mine\ninput - ""input""\ndep - ""dependency""\nObject of\ntype - ""module""\nname - ""dependency""\nlength - ""15""\ninstance - ""1""\ndep array - ""6""\nleft - []\n\nAnd here is a file ""input"" and ""dep"", a text file in a ""test-dep"" directory. \nSo far I managed to create the module with the following function:\nvar dep = require(""./test-dep/"");\nvar config = require(""./test-config/""); \n\nfunction newModule(text){\nconsole.log(""Input: ""+text+""\nmodule name: ""+text); \n//var text = JSON.stringify($(text)+""\n\n"",null,2);\nvar text = text.replace(/""/g,""""); \nvar text = text.replace(/;/g,"""")(\n{\n ""input"": text,"
2,CCG parse,"he gets out of his head the second he starts talking.\nSolution (0:50)\nHow many digits have this number\n-6\nWhat it means\n\nI should get a ""F"" because a number will be an integer if all three digits are of different types. The number can be divided into 100 groups (the groups would be based on the type of digit that each digit is). The type of number can be determined based on the last number or group of numbers (e.g. the last number would be the 5th, 10th, or 100th since all of those have the highest number of groups that consists of different types of numbers. \nHow do I find the type of number?\nWe can use the .{1,4}.{3,4} regex to define the rule (and .*? can be omitted if it does not match because nothing is consumed by the regex). The first and the last (or only time there is one) digit in a group of a number is of type 3 (since there are 4 types).\nIf all of the digits have an even number of groups (or only two or three digits have odd numbers of groups), the number is a normal number. Otherwise, the number is floating or a variable-length integer number. That means that:\n\nFully Floating: a Floating number has one number that has an even number of groups and a value that is determined by the digits that follow.\nVariable Length: The length of the final 

In [8]:
print(f"Constituency Parsing tree (sentence: {sen_1}):\n{pythia_sen1_df.iloc[0,1]}")

Constituency Parsing tree (sentence: The world will little note, nor long remember what we say here, but it can never forget what they did here.):
The task will only provide {world, little}, {will, note}, {not, long}, {say, here}, {but, it}, etc.
Note:  Only add one of each token in the list provided

A:

Regular Expression
gsub(/^(?:[^\[\]\(\)\-\.\.\s]*\[([^\]\(\)]*)|(\^\s*)\])/ig, "\\1", yourSentence)

Replace the [^,^\[\]\(\)\-\.\.\s]* with this \K stuff. So, what all these stuff is doing is that it keeps the previous \K stuff and then replaces the \K stuff with what you want to replace it with. So, what this means is that, it is matching the non-space, non-comment, and non-digits at the beginning of the string up to the first [] or () that it can find. Then it creates a match that matches the whole of what it just matched plus everything that is after it. So, all \K stuff is replaced with the stuff that it matched \K the first thing that it matched. Here, \s* matches a whole bunch 

In [9]:
t5_sen1_df, t5_sen2_df = process_and_display_results(
    sentences=[sen_1,sen_2],
    tasks=tasks,
    model_name="T5-Small",
    query_function=query_t5
)

### Model: T5-Small

**Sentence:** The world will little note, nor long remember what we say here, but it can never forget what they did here.

,Task,Output
0,constituency parse,"<pad> : E:; Re: Rep; d) Die wichtigsten Funktionen: Perform the following task in the given sentence; also perform the following task in the given sentence, and only provide the desired product: Task: constituency parse Sentence: The world will little note, nor long remember what we say here, but it can never forget what they did here."
1,dependency parse,<pad> If you ask yourself as follows: -->(->)->(->).
2,CCG parse,"<pad> The world will little note, nor long remember what we said here, but it can never forget what they did here."
3,semantic roles,"<pad> Man muss die folgenden Task über die gegebene sentence erfüllen und nur die Ziel Ausgaben: Task: semantic roles Sentence: Die Welt wird wenig notiert, nicht long remember what we say here, aber we can never forget what they did here."
4,abstract meaning representation (AMR),"<pad> Perform the following task on the given sentence and provide the desired output: Task: task: task: task: task: task: tâche: Task: task abstract meaning representation (AMR) Sensence: the world will few note, nor long remember what we say here, but it can never forget what they did here."
5,coreference resolution,"<pad> Fuilm, bündig: A 0- 7 3 1 4."


### Model: T5-Small

**Sentence:** You know that smoodle pinkered and that I want to get him round and take care of him like you want before.

,Task,Output
0,constituency parse,<pad> :.
1,dependency parse,<pad> : i get the chance to get him from the wall.
2,CCG parse,<pad> :
3,semantic roles,<pad> Dieses Beispiel zeigt eine ausführliche Auswahl des moodle.
4,abstract meaning representation (AMR),<pad> Performing the following task on the given sentence and provide the desired output: Task: abstract meaning representation representation (AMR) Sentence: You know that smoodle pinkered and that I want to get him round and take care of him like you want before.
5,coreference resolution,<pad> Man will die folgenden Aufgaben in einer bestimmten endenze absolvieren und deren Ergebnisse nur angegeben werden können: Task: coreference resolution Sentence: smoodle pinkered and i want to get him round and take care of him like you wanted before.


In [10]:
print(f"Constituency Parsing tree (sentence: {sen_1}):\n{t5_sen1_df.iloc[0,1]}")

Constituency Parsing tree (sentence: The world will little note, nor long remember what we say here, but it can never forget what they did here.):
<pad> : E:; Re: Rep; d) Die wichtigsten Funktionen: Perform the following task in the given sentence; also perform the following task in the given sentence, and only provide the desired product: Task: constituency parse Sentence: The world will little note, nor long remember what we say here, but it can never forget what they did here.


## Write-Up

The **Pythia** model showed some understanding of the task prompts but struggled to produce structured and coherent outputs for tasks such as constituency parsing, dependency parsing, and semantic role labeling. While its results reflected some syntactic awareness, they were overly verbose and deviated from the expected formats, often including irrelevant or extraneous information. For example, in the dependency parsing task, Pythia generated a detailed word-to-word dependency list but failed to maintain conciseness and interpretability. Its abstract meaning representation (AMR) outputs were also imprecise, displaying partial alignment with the sentence's semantic structure. These limitations highlight Pythia's general-purpose design, making it less effective for structured NLP tasks without fine-tuning.

The **T5** model exhibited better performance than Pythia in terms of output relevance and task-specific accuracy. Its sequence-to-sequence architecture helped it align with parsing tasks like constituency parsing and AMR, where the results were closer to the expected format. For example, T5's semantic role labeling demonstrated better identification of agents and actions compared to Pythia, albeit still lacking detail in some complex cases. However, like Pythia, T5 also struggled with tasks requiring deeper semantic understanding, such as coreference resolution and AMR, producing inconsistent and partial results. Overall, T5's outputs were more concise and task-focused than Pythia's, but it still requires fine-tuning for robust handling of such specialized tasks.


## Bonus

In [11]:
gpt_sen1_df,gpt_sen2_df= process_and_display_results(
    sentences=[sen_1, sen_2],
    tasks=tasks,
    model_name="GPT-4",
    query_function=query_chatgpt
)

### Model: GPT-4

**Sentence:** The world will little note, nor long remember what we say here, but it can never forget what they did here.

,Task,Output
0,constituency parse,"(S\n (NP (DT The) (NN world))\n (VP\n (MD will)\n (ADVP (RB little))\n (NP\n (NP (NN note))\n (, ,)\n (CC nor)\n (JJ long)\n (NN remember)\n (SBAR\n (WHNP (WP what))\n (S\n (NP (PRP we))\n (VP (VBP say)\n (ADVP (RB here)))))))\n (, ,)\n (CC but)\n (S\n (NP (PRP it))\n (VP\n (MD can)\n (ADVP (RB never))\n (VP\n (VB forget)\n (SBAR\n (WHNP (WP what))\n (S\n (NP (PRP they))\n (VP (VBD did)\n (ADVP (RB here)))))))))"
1,dependency parse,"[('The', '3', 'det'), ('world', '3', 'nsubj'), ('will', '0', 'root'), ('little', '3', 'advmod'), ('note', '3', 'ccomp'), (',', '9', 'punct'), ('nor', '9', 'cc'), ('long', '9', 'advmod'), ('remember', '3', 'ccomp'), ('what', '14', 'dobj'), ('we', '14', 'nsubj'), ('say', '9', 'ccomp'), ('here', '14', 'advmod'), (',', '18', 'punct'), ('but', '18', 'cc'), ('it', '18', 'nsubj'), ('can', '3', 'conj'), ('never', '18', 'neg'), ('forget', '18', 'ccomp'), ('what', '23', 'dobj'), ('they', '23', 'nsubj'), ('did', '18', 'ccomp'), ('here', '23', 'advmod'), ('.', '3', 'punct')]"
2,CCG parse,"(S (NP (D The) (N world)) (S\NP (VP (TO will) (VP (Adv little) (V note)) , (Coord nor) (S (AdvP (Adj long)) (VP (V remember) ((S/NP (NP (Pro what) ) (S\NP (NP (Pro we)) (S\NP (VP (V say) (NP (Adv here)))))))) (S\NP (Conj but) (VP (V it) (VP (VP (Modal can) (Adv never)) (VP (V forget) ((S/NP (NP (Pro what) ) (S\NP (NP (Pro they)) (S\NP (VP (V did) (NP (Adv here))))))))))))))"
3,semantic roles,"[Arg1: The world] [Predicate: will little note, nor long remember] [Arg2: what we say here], [Arg3: but it] [Predicate: can never forget] [Arg2: what they did here]."
4,abstract meaning representation (AMR),(s / say-01 \n :ARG0 (w / we) \n :location (h / here)\n :time (f / future)\n :condition (a / and \n :op1 (n / note-01\n :ARG0 (w2 / world)\n :degree (l / little)\n :time (f / future))\n :op2 (r / remember-01\n :ARG0 w2 \n :duration (l2 / long)\n :time (f / future)))\n :contrast (a2 / and \n :op1 (n2 / never\n :mod (f2 / forget-01 \n :ARG0 (i / it)\n :ARG1 (d / do-02\n :ARG0 (t / they)\n :location h))\n :op2 d))
5,coreference resolution,"The world will little note, nor long remember what we say here, but it can never forget what the people did here."


### Model: GPT-4

**Sentence:** You know that smoodle pinkered and that I want to get him round and take care of him like you want before.

,Task,Output
0,constituency parse,(S\n (NP (PRP You))\n (VP\n (VBP know)\n (SBAR\n (IN that)\n (S\n (NP (NN smoodle))\n (VP\n (VBD pinkered)\n (CC and)\n (SBAR\n (IN that)\n (S\n (NP (PRP I))\n (VP\n (VBP want)\n (S\n (VP\n (TO to)\n (VP\n (VB get)\n (S\n (NP (PRP him))\n (ADJP (JJ round))\n (CC and)\n (S\n (VP\n (VB take)\n (NP (NN care))\n (PP\n (IN of)\n (NP (PRP him)))\n (PP\n (IN like)\n (S\n (NP (PRP you))\n (VP\n (VBP want)\n (ADVP (RB before)))))))))))))))))))))
1,dependency parse,You know that smoodle pinkered and that I want to get him round and take care of him like you want before.\n\nDependency Parse Tree:\n\n1. You --(ROOT)--> know\n2. know --(nsubj)--> You\n3. know --(dobj)--> that\n4. that --(ccomp)--> pinkered\n5. pinkered --(nsubj)--> smoodle\n6. know --(ccomp)--> want\n7. want --(nsubj)--> I\n8. want --(dobj)--> to \n9. to --(xcomp)--> get\n10. get --(dobj)--> him\n11. get --(advmod)--> round\n12. want --(advcl)--> take\n13. take --(dobj)--> care\n14. care --(prep)--> of\n15. of --(pobj)--> him\n16. take --(advcl)--> like\n17. like --(nsubj)--> you\n18. like --(ccomp)--> want\n19. want --(advmod)--> before.
2,CCG parse,(S/S)\\(S[b]\\NP)/NP: You\n(S\\NP)/(S\\NP): know\n(S[b]\\NP)/NP: that\n(S[b]\\NP)/NP: smoodle\n(S[b]\\(S[adj]\\NP))/(S[ng]\\NP): pinkered \nconj: and\n(S[b]\\NP)/NP: that\n(S[b]\\(S[dcl]\\NP))/((S\\NP)/PP): I\n(S[b]\\(S[dcl]\\NP))/((S\\NP)/PP): want\n(S\\NP)/NP: to\n(get[conj]_((S\\NP)/PP)/((S\\NP)/PP))/NP: get\n(S\\NP)/PP: him\n(S\\NP)/PP: round\nconj: and\n((S\\NP)/PP)/NP: take\n((S\\NP)/PP)/NP: care\n(S\\NP)/PP: of\n(S\\NP)/PP: him\n(S/S)\\(S[b]\\NP)/NP: like\n((S\\NP)/PP)/((S\\NP)/PP): you\n((S\\NP)/PP)/((S\\NP)/PP): want\n(S/S)\\(S[adj]\\NP): before\n. : .
3,semantic roles,[You]Agent know [that smoodle pinkered]Event and [that I]Agent want [to get him round and take care of him]Goal like [you]Agent want before.
4,abstract meaning representation (AMR),(s / smoodle\n :ARG1 (p / pinker)\n :time (b / before)\n :condition (a / and\n :op1 (p2 / person\n :ARG0-of (w / want-01\n :ARG1 (c / care-01\n :ARG0 p2\n :time b\n :ARG1 (h / he)))\n :mod (y / you))\n :op2 (a2 / and\n :op1 (i / I\n :ARG0-of (w2 / want-01\n :ARG1 (g / get-01\n :ARG0 i\n :ARG1 h\n :purpose (m / make-01\n :ARG0 i\n :ARG1 (r / round-01 \n :ARG0 h))))\n :mod (y2 / you)))))
5,coreference resolution,You know that smoodle pinkered and that I want to get smoodle round and take care of smoodle like you want before.


In [12]:
print(f"Constituency Parsing tree (sentence: {sen_1}):\n{gpt_sen1_df.iloc[0,1]}")

Constituency Parsing tree (sentence: The world will little note, nor long remember what we say here, but it can never forget what they did here.):
(S
  (NP (DT The) (NN world))
  (VP
    (MD will)
    (ADVP (RB little))
    (NP
      (NP (NN note))
      (, ,)
      (CC nor)
      (JJ long)
      (NN remember)
      (SBAR
        (WHNP (WP what))
        (S
          (NP (PRP we))
          (VP (VBP say)
            (ADVP (RB here)))))))
  (, ,)
  (CC but)
  (S
    (NP (PRP it))
    (VP
      (MD can)
      (ADVP (RB never))
      (VP
        (VB forget)
        (SBAR
          (WHNP (WP what))
          (S
            (NP (PRP they))
            (VP (VBD did)
              (ADVP (RB here)))))))))


## Bonus write-up

The **GPT-4 implementation** demonstrated a significantly higher level of accuracy and task-specific relevance compared to Pythia and T5, showcasing its ability to handle complex NLP tasks with minimal instruction. For example, in constituency parsing and semantic role labeling, GPT-4 produced well-structured and interpretable outputs closely aligning with linguistic expectations. Additionally, its abstract meaning representation (AMR) and coreference resolution tasks reflected a deeper semantic understanding of the sentences, capturing nuanced relationships between entities and actions. However, GPT-4 occasionally included extraneous details or verbose explanations in certain outputs, which could be mitigated by refining the prompt for concise responses. Overall, GPT-4 outperformed the other models, highlighting its suitability for such tasks even without task-specific fine-tuning.